In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
## Loading model

model = load_model('Model/model.h5')

In [1]:
## Loading Encoder
import pickle
with open('Model/label_encoder.pkl', 'rb') as file :
  label_encoder_gender = pickle.load(file)

with open('Model/onehot_encoder.pkl', 'rb') as file :
  onehot_encoder_geography = pickle.load(file)

c:\Users\Palak\anaconda3\envs\newenv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Palak\anaconda3\envs\newenv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [30]:
## Loading Scaler

with open('Model/scaler.pkl', 'rb') as file :
  scaler = pickle.load(file)

In [31]:
## Example Input Data

input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [32]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [33]:
## One Hot Encoding of Geography Feature

geography_encoded = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geography_encode_df = pd.DataFrame(geography_encoded, columns = onehot_encoder_geography.get_feature_names_out(['Geography']))
geography_encode_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
## Label Encoding of Gender Feature

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [35]:
input_df = pd.concat([input_df.drop('Geography', axis = 1), geography_encode_df], axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
## Sacling input data

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [38]:
## Churn Prediction

prediction = model.predict(input_scaled)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[0.02131694]]


In [41]:
prediction_prob = prediction[0][0]
print(prediction_prob) ## Probability if a customer to churn

0.021316938


In [42]:
if prediction_prob > 0.5 :
  print('The Customer is likely to Churn')
else :
  print('The Customer is not likely to Churn')

The Customer is not likely to Churn
